In [1]:
import pandas as pd 


In [3]:
df = pd.read_csv('data/INFLUD25-07-07-2025.csv',sep = ';')

C:\Users\rafik\AppData\Local\Temp\ipykernel_26672\2456070258.py:1: DtypeWarning: Columns (14,92,94,185,186,188,189) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/INFLUD25-07-07-2025.csv',sep = ';')


In [4]:
df.head()

,NU_NOTIFIC,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,...,VG_OMS,VG_OMSOUT,VG_LIN,VG_MET,VG_METOUT,VG_DTRES,VG_ENC,VG_REINF,VG_CODEST,REINF
0,31735451969802,2024-12-29,1,2024-12-29,1,SP,GVE VIII MOGI DAS CRUZES,1333.0,GUARULHOS,351880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,31735516593521,2024-12-29,1,2024-12-29,1,SP,GVE VIII MOGI DAS CRUZES,1333.0,GUARULHOS,351880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,31735544021940,2024-12-29,1,2024-12-29,1,SP,GVE XXIX SAO JOSE DO RIO PRETO,1354.0,SAO JOSE DO RIO PRETO,354980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,31735559043222,2024-12-30,1,2024-12-29,1,AC,REGIONAL DO BAIXO ACRE,1938.0,RIO BRANCO,120040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,31735560267779,2024-12-30,1,2024-12-29,1,PB,I NRS JOAO PESSOA,1377.0,JOAO PESSOA,250750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [19]:
df_parcial = df[['DT_SIN_PRI','CLASSI_FIN', 'EVOLUCAO', 'HOSPITAL', 'UTI','VACINA_COV', 'DT_NOTIFIC','SG_UF', 'DT_EVOLUCA' ]].copy()

In [24]:
df_parcial

,DT_SIN_PRI,CLASSI_FIN,EVOLUCAO,HOSPITAL,UTI,VACINA_COV,DT_NOTIFIC,SG_UF,DT_EVOLUCA,data
2,2024-12-29,5.0,2.0,1.0,1.0,1.0,2024-12-29,SP,2025-01-21,2024-12-29
4,2024-12-29,5.0,1.0,1.0,2.0,1.0,2024-12-30,PB,2025-01-07,2024-12-29
6,2024-12-29,2.0,1.0,1.0,2.0,1.0,2024-12-29,SC,2025-01-10,2024-12-29
7,2024-12-29,2.0,2.0,1.0,1.0,1.0,2024-12-30,MS,2025-01-27,2024-12-29
8,2024-12-29,4.0,1.0,1.0,1.0,1.0,2024-12-30,SP,2024-12-31,2024-12-29
...,...,...,...,...,...,...,...,...,...,...
180087,2025-06-16,4.0,2.0,1.0,1.0,2.0,2025-06-18,MA,2025-06-19,2025-06-16
180092,2025-06-05,1.0,1.0,1.0,2.0,2.0,2025-06-21,SP,2025-06-30,2025-06-05
180094,2025-06-29,1.0,1.0,1.0,2.0,2.0,2025-07-01,SP,2025-07-04,2025-06-29
180096,2025-06-06,4.0,2.0,1.0,2.0,1.0,2025-07-06,SP,2025-07-02,2025-06-06


In [20]:
df_parcial = df_parcial.dropna()

In [27]:
df_parcial['DT_SIN_PRI'] = pd.to_datetime(df_parcial['DT_SIN_PRI'], format='%d/%m/%Y', errors='coerce')
df_parcial['DT_NOTIFIC'] = pd.to_datetime(df_parcial['DT_NOTIFIC'], format='%d/%m/%Y', errors='coerce')
df_parcial['DT_EVOLUCA'] = pd.to_datetime(df_parcial['DT_EVOLUCA'], format='%d/%m/%Y', errors='coerce')

In [28]:
# Criar coluna data base (caso DT_SIN_PRI falte)
df_parcial["data"] = df_parcial["DT_SIN_PRI"].fillna(df_parcial["DT_NOTIFIC"])

# Criar colunas booleanas
df_parcial["is_obito"] = df_parcial["EVOLUCAO"] == 2
df_parcial["is_uti"] = df_parcial["UTI"] == 1
df_parcial["is_hospitalizado"] = df_parcial["HOSPITAL"] == 1
df_parcial["is_vacinado"] = df_parcial["VACINA_COV"] == 1

# Filtrar últimos 30 dias (ajuste a data conforme necessário)
df_parcial = df_parcial[df_parcial["data"] >= pd.Timestamp.today() - pd.Timedelta(days=30)]

# Agregar por data
df_agg = df_parcial.groupby("data").agg(
    total_casos=("data", "count"),
    obitos=("is_obito", "sum"),
    hospitalizados=("is_hospitalizado", "sum"),
    uti=("is_uti", "sum"),
    vacinados=("is_vacinado", "sum")
).reset_index()

# Calcular média móvel
df_agg["media_movel_7d"] = df_agg["total_casos"].rolling(7).mean()

# Salvar para CSV
#df_agg.to_csv("srag_diario.csv", index=False)

In [29]:
df_agg

,data,total_casos,obitos,hospitalizados,uti,vacinados,media_movel_7d


In [21]:
df_parcial.to_csv(('data/srag_parcial.csv'), index=False, sep=';')